In [2]:
#Importar librerias
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sympy import diff,symbols,dsolve, expand_complex,Matrix, pi, cos, sin, simplify, eye, solve, latex, atan2, pprint, init_printing, Derivative, sqrt, integrate, Eq, lambdify
from sympy.physics.mechanics import ReferenceFrame, dynamicsymbols, init_vprinting

# Configuración de impresión en formato LaTeX (MathJax)
init_vprinting(use_latex='mathjax')

In [3]:
theta, x = dynamicsymbols("theta, x")
t,m,l,g = symbols("t,m,l,g")
N = ReferenceFrame("N") 
A = N.orientnew("A", "Axis", [pi/2 - theta, N.z])
A.x.express(N).subs({"theta":np.deg2rad(15)})

#hallar CM, como es en una línea es sólo en un promedio ponderado directo
x_cm = (m*l+m*l/2)/(2*m)

rcm = -x_cm*A.x + x*N.x

#sum F = m*a_total, la relativa es 0 ya que no rota pero respecto al mundo sí se está acelerado
#El del X sí se está acelerando, para eso hacemos un vector de O a P, uno que se mueva a la derecha
vel = rcm.diff(t,N)
a_cm = vel.diff(t,N)

#Sumatoria de fuerzas, sumaria de momentos
F12_x, F12_y = symbols("F12_x, F12_y")
F12 = F12_x*N.x + F12_y*N.y

eqF = F12 - m*g*N.y - m*a_cm #Sum fuerza = m*acm ==> 0 = sum fuerzas - m*acm
Icm_bola = m*(l/4)**2 #l/2 - l/4
Icm_palo_mitad = (m*l**2)/12 
Icm_palo_cm = Icm_palo_mitad + (m*(l/4)**2) #correr la inercia del palo, a la inercia del centro de masa

Icm = Icm_bola + Icm_palo_cm

eqM = rcm*A.x.cross(F12) - Icm*(-theta.diff(t,N)*diff(t,N))
#es negativo porque Momento positivo es que el theta se disminuya, por las convenciones de momentos sentido antihorario y ángulo crece horario
eqs = [eqF.dot(N.x), eqF.dot(N.y), eqM.dot(N.z)]
eqs_num = [eq.subs({m:1,l:1, g:9.81 }) for eq in eqs]

sln = solve(eqs_num, [theta,diff(t,t), F12_x,F12_y])
sln[theta.diff(t,t)] #queda integrar para saber cómo cambia

SympifyError: SympifyError: "cannot sympify object of type <class 'sympy.physics.vector.vector.Vector'>"

In [ ]:
#En este ejercicio se quiere observar cómo cambia la velocidad angular (la cual el enunciado nos dice que era constante)
#Debido al añadir un momento de 30N*m

#Marcos de referencia, tiempo, longitudes, masas y angulo
#vamos a medir las bolas de arriba hacia abajo, la vertical es 1, la horizontal es 2 y la lateral derecha abajo es 3
N = ReferenceFrame("N") 
l1, l2, l3, m1, m2, m3, t, g, M_externo = symbols("l1, l2, l3, m1, m2, m3, t, g, M_externo")
theta = dynamicsymbols("theta")
A = N.orientnew("A", "Axis", [-pi + theta, N.z])

#Creamos el vector del centro de masa, a partir de los vectores posicion de las esferas
p_1 = l1*(N.y.express(A))
p_2 = -l2*(N.x.express(A))
p_3 = l1*A.x
vector_cm = (m1*p_1 + m2*p_2 + m3*p_3)/(m1+m2+m3)
vel = vector_cm.diff(t,N)
a_cm = vel.diff(t,N)

#Hallamos fuerzas:
#Notamos que hay fuerzas por peso de cada bola y sus respectivas reacciones
#Las reacciones tienen el mismo sentido que las posiciones, solo que con un menos al ser en sentido contrario

#Hallamos inercia, como estamos en el centro de masa, la inercia es solo por teorema de ejes paralelos
I_1 = (vector_cm - p_1).magnitude()**2 * m1 
I_2 = (vector_cm - p_2).magnitude()**2 * m2 
I_3 = (vector_cm - p_3).magnitude()**2 * m3 
I = I_1 + I_2 + I_3


# --- Definir las componentes de las reacciones (dos por reacción en el plano) ---
Rx, Ry = symbols("R1x R1y")
R = Rx*N.x + Ry*N.y

# --- Pesos (igual que antes) ---
peso_1 = -m1*g*N.y
peso_2 = -m2*g*N.y
peso_3 = -m3*g*N.y

# --- Ecuación de fuerzas (ΣF = m a_cm) ---
eqF = R+ peso_1 + peso_2 + peso_3 - (m1+m2+m3)*a_cm

# --- Momentos respecto al centro de masa (Σ (r_i - r_cm) × F_i + M_externo = I*α) ---
# Nota: usa p_i - vector_cm como brazo y cruza con la fuerza aplicada en p_i
M_pesos = (p_1 - vector_cm).cross(peso_1) + (p_2 - vector_cm).cross(peso_2) + (p_3 - vector_cm).cross(peso_3)
M_reacs = (vector_cm).cross(R)

eqM = M_pesos + M_reacs + M_externo*N.z - I*theta.diff(t,2)*N.z

# --- Ecuaciones escalares relevantes ---
eqs = [eqF.dot(N.x), eqF.dot(N.y), eqM.dot(N.z)]

# --- Sustituir datos numéricos como antes ---
dicti = { l1:0.5, l2:0.4, l3:0.6, m1:3, m2:4, m3:3, M_externo:30, g:9.81 }
eqs_num = [simplify(eq.subs(dicti)) for eq in eqs]

# --- Intentar resolver ---
# Incógnitas: theta'' y las componentes de reacción
unknowns = [theta.diff(t,2), Rx, Ry]
sln = solve(eqs_num, unknowns, dict=True)[0][theta.diff(t,2)].simplify()
sln